In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from optiver_features import *

In [16]:
#df_train = pd.read_csv('train_with_features.csv')
df_train = generate_train_df()

In [17]:
df_train.columns.to_list()

['stock_id',
 'time_id',
 'target',
 'row_id',
 'wap1_sum',
 'wap1_mean',
 'wap1_std',
 'wap2_sum',
 'wap2_mean',
 'wap2_std',
 'log_return1_sum',
 'log_return1_realized_volatility',
 'log_return1_mean',
 'log_return1_std',
 'log_return2_sum',
 'log_return2_realized_volatility',
 'log_return2_mean',
 'log_return2_std',
 'wap_balance_sum',
 'wap_balance_mean',
 'wap_balance_std',
 'price_spread_sum',
 'price_spread_mean',
 'price_spread_std',
 'bid_spread_sum',
 'bid_spread_mean',
 'bid_spread_std',
 'ask_spread_sum',
 'ask_spread_mean',
 'ask_spread_std',
 'total_volume_sum',
 'total_volume_mean',
 'total_volume_std',
 'volume_imbalance_sum',
 'volume_imbalance_mean',
 'volume_imbalance_std',
 'wap1_sum_450',
 'wap1_mean_450',
 'wap1_std_450',
 'wap2_sum_450',
 'wap2_mean_450',
 'wap2_std_450',
 'log_return1_sum_450',
 'log_return1_realized_volatility_450',
 'log_return1_mean_450',
 'log_return1_std_450',
 'log_return2_sum_450',
 'log_return2_realized_volatility_450',
 'log_return2_mea

## TODO: fill with 0 or median depending on the column

In [18]:
df_train = df_train.fillna(0)

In [19]:
kfold = KFold(n_splits = 5, random_state = 66, shuffle = True)


In [20]:
df_train.stock_id = df_train.stock_id.astype('category')

In [21]:
cont_nn,cat_nn = cont_cat_split(df_train, max_card=9000, dep_var='target')

In [22]:
cat_nn.remove('time_id'), cat_nn.remove('row_id')

(None, None)

In [23]:
def train_fold(trn_idx, val_idx, fname):
    

    procs_nn = [Categorify, Normalize]
    to_nn = TabularPandas(df_train, procs_nn, cat_nn, cont_nn,
                          splits=[list(trn_idx), list(val_idx)], y_names='target')

    dls = to_nn.dataloaders(1024)
    learn = tabular_learner(dls, y_range=(0,.1), layers=[1000,500,200],config={'lin_first':False},
                        n_out=1, loss_func=rmspe_loss, metrics=AccumMetric(rmspe),
                        #cbs=[EarlyStoppingCallback(monitor='rmspe', comp=np.less, patience=20),
                        #    SaveModelCallback(monitor='rmspe', comp=np.less, fname=fname)]
                           )
    learn.fit_one_cycle(70, 5e-3)
    score = learn.recorder.metrics[0].value
    #score = learn.early_stopping.best
    learn.save(fname)
    return score

In [24]:
def rmspe(preds, targs):
    x = (targs-preds)/targs
    return (x**2).mean().sqrt()

In [25]:
def rmspe_loss(preds, targs):
    x = (targs-preds)/targs
    return (x**2).mean().sqrt()

In [ ]:
res=[]
for trn_idx, val_idx in kfold.split(df_train):
    res.append(train_fold(trn_idx, val_idx, f'def_layers_fold_{len(res)}'))
    print('fold', len(res), res[-1])
np.mean(res),res

epoch,train_loss,valid_loss,rmspe,time
0,0.273089,0.252424,0.259620,00:04
1,0.252365,0.240490,0.243253,00:03
2,0.250641,0.249488,0.253966,00:03
3,0.251349,0.255299,0.262647,00:03
4,0.248811,0.236007,0.238366,00:03
5,0.260909,0.268782,0.271035,00:03
6,0.246353,0.252145,0.254460,00:03
7,0.249560,0.250110,0.252475,00:03
8,0.243495,0.254405,0.256738,00:03
9,0.248523,0.250132,0.252179,00:03


fold 1 TensorBase(0.2292)


epoch,train_loss,valid_loss,rmspe,time
0,0.268924,0.239300,0.241491,00:03
1,0.253232,0.235212,0.237632,00:03
2,0.252849,0.239887,0.242382,00:03
3,0.252736,0.238097,0.242544,00:03
4,0.250096,0.237078,0.240207,00:03
5,0.255375,0.244118,0.245853,00:03
6,0.253511,0.261416,0.266612,00:03
7,0.245724,0.261320,0.263496,00:03
8,0.260551,0.239531,0.242147,00:03
9,0.249955,0.239266,0.241265,00:03


fold 2 TensorBase(0.2262)


epoch,train_loss,valid_loss,rmspe,time
0,0.271965,0.248548,0.251353,00:03
1,0.259481,0.243462,0.247404,00:03
2,0.247990,0.240145,0.242072,00:03
3,0.248268,0.242757,0.245261,00:03
4,0.253924,0.240406,0.242590,00:03
5,0.254179,0.248412,0.250612,00:03
6,0.251207,0.254357,0.257064,00:03
7,0.250100,0.251106,0.254768,00:03
8,0.249454,0.235486,0.237184,00:03
9,0.254027,0.261218,0.266941,00:03


fold 3 TensorBase(0.2292)


epoch,train_loss,valid_loss,rmspe,time
0,0.270646,0.246036,0.251729,00:03
1,0.252404,0.256137,0.263354,00:03
2,0.249952,0.274667,0.281868,00:03
3,0.246872,0.241262,0.249649,00:03
4,0.246704,0.238290,0.242095,00:03
5,0.254688,0.263187,0.268333,00:03
6,0.259048,0.249166,0.254527,00:03
7,0.247456,0.232825,0.235936,00:03
8,0.246005,0.244787,0.246898,00:03
9,0.243740,0.233039,0.235290,00:03


fold 4 TensorBase(0.2336)


epoch,train_loss,valid_loss,rmspe,time
0,0.270269,0.244844,0.247133,00:03
1,0.256700,0.238366,0.240932,00:03
2,0.246895,0.236224,0.239196,00:03
3,0.253896,0.243661,0.246520,00:03
4,0.256531,0.237742,0.240337,00:03
5,0.250032,0.235504,0.239005,00:03
6,0.250671,0.255950,0.258103,00:03
7,0.247844,0.237165,0.238818,00:03
8,0.255419,0.249497,0.251523,00:03
9,0.246456,0.235901,0.238413,00:03
